# Introduction to Spark lab, part 1: Basic concepts
This notebook guides you through the basic concepts to start working with Spark, including how to set up your environment, create and analyze data sets, and work with data files.

This notebook uses pySpark, the Python API for Spark. Some knowledge of Python is recommended. This notebook runs on Python and Spark.

If you are new to notebooks, here's how the user interface works: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/parts-of-a-notebook.html" target="_blank" rel="noopener noreferrer">The parts of a notebook</a>.

## About Spark
Spark is a fast and general-purpose cluster computing system. It provides high-level APIs in Java, Scala, Python and R, and an optimized engine that supports general execution graphs. It also supports a rich set of higher-level tools including Spark SQL for processing structured data, MLlib for machine learning, GraphX for graph processing, and Spark Streaming.

<img src='https://github.com/carloapp2/SparkPOT/blob/master/spark.png?raw=true' width="50%" height="50%"></img>


A Spark program has a driver program and worker programs. Worker programs run on cluster nodes or in local threads. Data sets are distributed across workers. 

<img src='https://github.com/carloapp2/SparkPOT/blob/master/Spark%20Architecture.png?raw=true' width="50%" height="50%"></img>

## Table of Contents
In the first four sections of this notebook, you'll learn about Spark with very simple examples. In the last two sections, you'll use what you learned to analyze data files that have more realistic data sets.

1. [Work with the SparkContext](#sparkcontext)<br>
    1.1 [Invoke the SparkContext](#sparkcontext1)<br>
    1.2 [Check the Spark version](#sparkcontext2)<br>
2. [Work with RDDs](#rdd)<br>
    2.1 [Create a collection](#rdd1)<br>
    2.2 [Create an RDD](#rdd2)<br>
    2.3 [View the data](#rdd3)<br>
    2.4 [Create another RDD](#rdd4)<br>
3. [Manipulate data in RDDs](#trans)<br>
    3.1 [Update numeric values](#trans1)<br>
    3.2 [Add numbers in an array](#trans2)<br>
    3.3 [Split and count strings](#trans3)<br>
    3.4 [Counts words with a pair RDD](#trans4)<br>
4. [Filter data](#filter)<br>
5. [Analyze text data from a file](#wordfile)<br>
    5.1 [Get the data from a URL](#wordfile1)<br>
    5.2 [Create an RDD from the file](#wordfile2)<br>
    5.3 [Filter for a word](#wordfile3)<br>
    5.4 [Count instances of a string at the beginning of words](#wordfile4)<br>
    5.5 [Count instances of a string within words](#wordfile5)<br>
6. [Analyze numeric data from a file](#numfile)<br>
7. [Summary and next steps](#summary)

<a id="sparkcontext"></a>
## 1. Work with the SparkContext object

The Spark driver application uses the SparkContext object to allow a programming interface to interact with the driver application. The SparkContext object tells Spark how and where to access a cluster.

The Watson Studio notebook environment predefines the Spark context for you.

In other environments, you need to pick an interpreter (for example, pyspark for Python) and create a SparkConf object to initialize a SparkContext object. For example:
<br>
`from pyspark import SparkContext, SparkConf`<br>
`conf = SparkConf().setAppName(appName).setMaster(master)`<br>
`sc = SparkContext(conf=conf)`<br>

<a id="sparkcontext1"></a>
### 1.1 Invoke the SparkContext
Run the following cell to invoke the SparkContext:

In [1]:
sc

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190629194315-0000
KERNEL_ID = 567fa8f4-5d74-4e3f-9714-c0fcaf01dd3f


<SparkContext master=spark://jkg-deployment-567fa8f4-5d74-4e3f-9714-c0fcaf01dd3f-69556fk2v4r:7077 appName=pyspark-shell>

<a id="sparkcontext2"></a>
### 1.2 Check the Spark version
Check the version of the Spark driver application:

In [2]:
sc.version

'2.3.3'

<a id="rdd"></a>
## 2. Work with Resilient Distributed Datasets
Spark uses an abstraction for working with data called a Resilient Distributed Dataset (RDD). An RDD is a collection of elements that can be operated on in parallel. RDDs are immutable, so you can't update the data in them. To update data in an RDD, you must create a new RDD. In Spark, all work is done by creating new RDDs, transforming existing RDDs, or using RDDs to compute results. When working with RDDs, the Spark driver application automatically distributes the work across the cluster.

You can construct RDDs by parallelizing existing Python collections (lists), by manipulating RDDs, or by manipulating files in HDFS or any other storage system.

You can run these types of methods on RDDs: 
 - Actions: query the data and return values
 - Transformations: manipulate data values and return pointers to new RDDs. 

Find more information on Python methods in the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.html" target="_blank" rel="noopener noreferrer">PySpark documentation</a>.

<a id="rdd1"></a>
### 2.1 Create a collection
Create a Python collection of the numbers 1 - 10:

In [3]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

<a id="rdd2"></a>
### 2.2 Create an RDD 
Put the collection into an RDD named `x_nbr_rdd` using the `parallelize` method:

In [4]:
x_nbr_rdd = sc.parallelize(x)

Notice that there's no return value. The `parallelize` method didn't compute a result, which means it's a transformation. Spark just recorded how to create the RDD.

<a id="rdd3"></a>
### 2.3 View the data 
View the first element in the RDD:

In [5]:
x_nbr_rdd.first()

1

Each number in the collection is in a different element in the RDD. Because the `first()` method returned a value, it is an action. 

Now view the first five elements in the RDD:

In [6]:
x_nbr_rdd.take(5)

[1, 2, 3, 4, 5]

<a id="rdd4"></a>
### 2.4 Create another RDD 
Create a Python collection that contains strings:

In [7]:
y = ["Hello Human", "My Name is Spark"]

Put the collection into an RDD:

In [8]:
y_str_rdd = sc.parallelize(y)

View the first element in the RDD:

In [9]:
y_str_rdd.take(1)

['Hello Human']

You created the string "Hello Human" and you returned it as the first element of the RDD. To analyze a set of words, you can map each word into an RDD element.

<a id="trans"></a>
## 3. Manipulate data in RDDs

Remember that to manipulate data, you use transformation functions.

Here are some common Python transformation functions that you'll be using in this notebook:

 - `map(func)`: returns a new RDD with the results of running the specified function on each element  
 - `filter(func)`: returns a new RDD with the elements for which the specified function returns true   
 - `distinct([numTasks]))`: returns a new RDD that contains the distinct elements of the source RDD
 - `flatMap(func)`: returns a new RDD by first running the specified function on all elements, returning 0 or more results for each original element, and then flattening the results into individual elements

You can also create functions that run a single expression and don't have a name with the Python `lambda` keyword. For example, this function returns the sum of its arguments: `lambda a , b : a + b`.

<a id="trans1"></a>
### 3.1 Update numeric values
Run the `map()` function with the `lambda` keyword to replace each element, X, in your first RDD (the one that has numeric values) with X+1. Because RDDs are immutable, you need to specify a new RDD name.

In [10]:
x_nbr_rdd_2 = x_nbr_rdd.map(lambda x: x+1)

Now look at the elements of the new RDD: 

In [11]:
x_nbr_rdd_2.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Be careful with the `collect` method! It returns __all__ elements of the RDD to the driver. Returning a large data set might be not be very useful. No-one wants to scroll through a million rows!

<a id="trans2"></a>
### 3.2 Add numbers in an array
An array of values is a common data format where multiple values are contained in one element. You can manipulate the individual values if you split them up into separate elements.

Create an array of numbers by including quotation marks around the whole set of numbers. If you omit the quotation marks, you get a collection of numbers instead of an array.

In [12]:
X = ["1,2,3,4,5,6,7,8,9,10"]

Create an RDD for the array:

In [13]:
y_rd = sc.parallelize(X)

Split the values at commas and add values in the positions 2 and 9 in the array. Keep in mind that an array starts with position 0. Use a backslash character, \, to break the line of code for clarity.

In [14]:
Sum_rd = y_rd.map(lambda y: y.split(",")).\
map(lambda y: (int(y[2])+int(y[9])))

Now return the value of the sum:

In [15]:
Sum_rd.first()

13

<a id="trans3"></a>
### 3.3 Split and count text strings

Create an RDD with a text string and show the first element:

In [16]:
Words = ["Hello Human. I'm Spark and I love running analysis on data."]
words_rd = sc.parallelize(Words)
words_rd.first()

"Hello Human. I'm Spark and I love running analysis on data."

Split the string into separate lines at the space characters and look at the first element:

In [17]:
Words_rd2 = words_rd.map(lambda line: line.split(" "))
Words_rd2.first()

['Hello',
 'Human.',
 "I'm",
 'Spark',
 'and',
 'I',
 'love',
 'running',
 'analysis',
 'on',
 'data.']

Count the number of elements in this RDD with the `count()` method:

In [18]:
Words_rd2.count()

1

Of course, you already knew that there was only one element because you ran the `first()` method and it returned the whole string. Splitting the string into multiple lines did not create multiple elements.

Now split the string again, but this time with the `flatmap()` method, and look at the first three elements:

In [19]:
words_rd2 = words_rd.flatMap(lambda line: line.split(" "))
words_rd2.take(3)

['Hello', 'Human.', "I'm"]

This time each word is separated into its own element.

<a id="trans4"></a>
### 3.4 Count words with a pair RDD
A common way to count the number of instances of words in an RDD is to create a pair RDD. A pair RDD converts each word into a key-value pair: the word is the key and the number 1 is the value. Because the values are all 1, when you add the  values for a particular word, you get the number of instances of that word.

Create an RDD:

In [20]:
z = ["First,Line", "Second,Line", "and,Third,Line"]
z_str_rdd = sc.parallelize(z)
z_str_rdd.first()

'First,Line'

Split the elements into individual words with the `flatmap()` method:

In [21]:
z_str_rdd_split_flatmap = z_str_rdd.flatMap(lambda line: line.split(","))
z_str_rdd_split_flatmap.collect()

['First', 'Line', 'Second', 'Line', 'and', 'Third', 'Line']

Convert the elements into key-value pairs:

In [22]:
countWords = z_str_rdd_split_flatmap.map(lambda word:(word,1))
countWords.collect()

[('First', 1),
 ('Line', 1),
 ('Second', 1),
 ('Line', 1),
 ('and', 1),
 ('Third', 1),
 ('Line', 1)]

Now sum all the values by key to find the number of instances for each word: 

In [23]:
from operator import add
countWords2 = countWords.reduceByKey(add)
countWords2.collect()

[('Second', 1), ('Line', 3), ('First', 1), ('and', 1), ('Third', 1)]

Notice that the word `Line` has a count of 3.

<a id="filter"></a>
## 4. Filter data

The filter command creates a new RDD from another RDD based on a filter criteria.
The filter syntax is: 

`.filter(lambda line: "Filter Criteria Value" in line)`

Hint: Use a simple python `print` command to add a string to your Spark results and to run multiple actions in single cell.

Find the number of instances of the word `Line` in the `z_str_rdd_split_flatmap` RDD:

In [24]:
words_rd3 = z_str_rdd_split_flatmap.filter(lambda line: "Second" in line) 

print ("The count of words " + str(words_rd3.first()))
print ("Is: " + str(words_rd3.count()))

The count of words Second
Is: 1


<a id="wordfile"></a>
## 5. Analyze text data from a file
In this section, you'll download a file from a URL, create an RDD from it, and analyze the text in it.

<a id="wordfile1"></a>
### 5.1 Get the file from a URL

You can run shell commands by prefacing them with an exclamation point (!).

Remove any files with the same name as the file that you're going to download and then load a file named `README.md` from a URL into the filesystem for Spark:

In [25]:
!rm README.md* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md

--2019-06-29 19:45:33--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3624 (3.5K) [text/plain]
Saving to: 'README.md'

README.md           100%[===================>]   3.54K  --.-KB/s    in 0s      

2019-06-29 19:45:33 (96.7 MB/s) - 'README.md' saved [3624/3624]



<a id="wordfile2"></a>
### 5.2 Create an RDD from the file
Use the `textFile` method to create an RDD named `textfile_rdd` based on the `README.md` file. The RDD will contain one element for each line in the `README.md` file.
Also, count the number of lines in the RDD, which is the same as the number of lines in the text file. 

In [26]:
textfile_rdd = sc.textFile("README.md")
textfile_rdd.count()

98

<a id="wordfile3"></a>
### 5.3 Filter for a word 
Filter the RDD to keep only the elements that contain the word "Spark" with the `filter` transformation:

In [27]:
Spark_lines = textfile_rdd.filter(lambda line: "Spark" in line)
Spark_lines.first()

'# Apache Spark'

Count the number of elements in this filtered RDD and present the result as a concatenated string:

In [28]:
print ("The file README.md has " + str(Spark_lines.count()) + \
" of " + str(textfile_rdd.count()) + \
" Lines with word Spark in it.")

The file README.md has 19 of 98 Lines with word Spark in it.


<a id="wordfile4"></a>
### 5.4 Count the instances of a string at the beginning of words
Count the number of times the substring "Spark" appears at the beginning of a word in the original text.

Here's what you need to do: 

1. Run a `flatMap` transformation on the Spark_lines RDD and split on white spaces.
2. Create an RDD with key-value pairs where the first element of the tuple is the word and the second element is the number 1.
3. Run a `reduceByKey` method with the `add` function to count the number of instances of each word.<br>
4. Filter the resulting RDD to keep only the elements that start with the word "Spark". In Python, the syntax to determine whether a string starts with a token is: `string.startswith("token")` 
5. Display the resulting list of elements that start with "Spark".

In [29]:
temp = Spark_lines.flatMap(lambda line:line.split(" ")).map(lambda x:(x,1)).reduceByKey(add)
temp.filter(lambda x: x[0].startswith("Spark")).collect()

[('SparkPi', 2),
 ('Spark](#building-spark).', 1),
 ('Spark', 14),
 ('Spark.', 1),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1)]

<a id="wordfile5"></a>
### 5.5 Count instances of a string within words
Now filter and display the elements that contain the substring "Spark" anywhere in the word, instead of just at the beginning of words like the last section. Your result should be a superset of the previous result.

The Python syntax to determine whether a string contains a particular token is: `"token" in string`

In [30]:
temp.filter(lambda x: "Spark" in x[0]).collect()

[('SparkPi', 2),
 ('Spark](#building-spark).', 1),
 ('Spark', 14),
 ('Spark.', 1),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('tests](https://cwiki.apache.org/confluence/display/SPARK/Contributing+to+Spark#ContributingtoSpark-AutomatedTesting).',
  1)]

<a id="numfile"></a>
## 6. Analyze numeric data from a file
You'll analyze a sample file that contains instructor names and scores. The file has the following format: Instructor Name,Score1,Score2,Score3,Score4. 
Here is an example line from the text file: "Carlo,5,3,3,4"

Add all scores and report on results:

1. Download the file.
1. Load the text file into an RDD.
1. Run a transformation to create an RDD with the instructor names and the sum of the 4 scores per instructor.
1. Run a second transformation to compute the average score for each instructor.
1. Display the first 5 results.

In [31]:
!rm Scores.txt* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
 
Raw_Rdd = sc.textFile("Scores.txt")

SumScores = Raw_Rdd.map(lambda l: l.split(",")).\
map(lambda v : (v[0], int(v[1])+int(v[2])+int(v[3])+int(v[4])))

Final = SumScores.map(lambda avg: (avg[0],avg[1],avg[1]/4.0))

Final.take(5)

--2019-06-29 19:46:07--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75 [text/plain]
Saving to: 'Scores.txt'

Scores.txt          100%[===================>]      75  --.-KB/s    in 0s      

2019-06-29 19:46:08 (12.8 MB/s) - 'Scores.txt' saved [75/75]



[('Carlo', 15, 3.75),
 ('Mokhtar', 15, 3.75),
 ('Jacques', 15, 3.75),
 ('Braden', 15, 3.75),
 ('Chris', 15, 3.75)]

<a id="summary"></a>
## 7. Summary and next steps

You've learned how to work with data in RDDs to discover useful information.

Look for the other notebooks in this series: 
 - <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/5ad1c820f57809ddec9a040e37b2bd55" target="_blank" rel="noopener noreferrer">Introduction to Spark lab, part 2: Querying data</a>
 - <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/5ad1c820f57809ddec9a040e37b4af08" target="_blank" rel="noopener noreferrer">Introduction to Spark lab, part 3: Machine learning</a>

Dig deeper:
 - <a href="http://spark.apache.org/documentation.html" target="_blank" rel="noopener noreferrer">Apache Spark documentation</a>
 - <a href="http://spark.apache.org/docs/latest/api/python/pyspark.html" target="_blank" rel="noopener noreferrer">PySpark documentation</a>

### Authors
**Carlo Appugliese** is a Spark and Hadoop evangelist at IBM.<br>
**Braden Callahan** is a Big Data Technical Specialist for IBM.<br>
**Ross Lewis** is a Big Data Technical Sales Specialist for IBM.<br>
**Mokhtar Kandil** is a World Wide Big Data Technical Specialist for IBM.

<hr>
Copyright &copy; IBM Corp. 2017-2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>